In [5]:
search_for = 121
start_from = 0
threads = 10
thread = 8
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.004076480865478516
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 15
all cases: 237758115
type: [1, 1, 1, 1, 121] 121
cases of this type: 214358881
100000 9.296870812088017
200000 17.64775988335108
300000 18.389353765419152
400000 18.079724682818863
500000 18.91851704408701
600000 17.545669698535352
700000 18.606966263115027
800000 18.444156102382568
900000 10.196645160461335
1000000 14.570986811027064
1100000 19.41502988350267
1200000 18.913991163744775
1300000 18.8510323502069
1400000 19.915019489701933
1500000 17.394264240134525
1600000 17.9088149882393
1700000 17.589361017885313
1800000 8.622224163623711
1900000 16.23734698884716
2000000 22.60445725935818
2100000 22.346057785492214
2200000 21.0464874028083
2300000 21.237386270415442
2400000 20.182764142192344
2500000 19.38295047148057
2600000 19.739537382205736
2700000 7.778549388967623
2800000 20.587211017245252
2900000 20.404746728271384
3000000 20.18

29400000 13.856797722282431
29500000 14.989628304650456
29600000 15.641921109727921
29700000 15.674393882677933
29800000 16.265296399095828
29900000 9.073243485504053
30000000 15.266296228180288
30100000 15.398998539806353
30200000 15.497544814561229
30300000 15.32211518251708
30400000 9.50282294635674
30500000 15.190365696236123
30600000 15.515461356273153
30700000 15.521631670470967
30800000 14.427335498320764
30900000 14.215430150630981
31000000 9.801127963812256
31100000 10.30246549587243
31200000 13.741607258199144
31300000 15.068073580338005
31400000 15.532191784765402
31500000 15.377956856816859
31600000 16.597309868137774
31700000 8.564908108450968
31800000 15.685701108266949
31900000 16.077307563456383
32000000 15.728291730625692
32100000 15.869499409162682
32200000 7.6538616072993575
32300000 16.557981286714742
32400000 15.216426550135713
32500000 14.690588446587112
32600000 15.891235407989814
32700000 13.371441773993702
32800000 6.923887577750654
32900000 13.03196274902143
3

59000000 6.1287563581799835
59100000 12.04823540833072
59200000 12.862753702682916
59300000 12.117482820793155
59400000 4.952429133398353
59500000 12.248269613896822
59600000 11.93987068755255
59700000 12.301877996137081
59800000 6.682668240786874
59900000 12.469906867074817
60000000 12.7779181264821
60100000 13.590370909693533
60200000 13.250935204141443
60300000 13.538882174600861
60400000 13.770876370052695
60500000 13.012607982278926
60600000 12.29566449965644
60700000 12.756335712886932
60800000 5.878933635675702
60900000 11.960910261617958
61000000 12.54654968091939
61100000 10.323702251279085
61200000 7.1149606048787435
61300000 12.347565903498491
61400000 11.97977438835882
61500000 9.099715554474235
61600000 9.978252657518334
61700000 12.927987683379845
61800000 12.547394703457584
61900000 12.599153904631045
62000000 12.914853087920473
62100000 13.378267647520115
62200000 13.372434643702212
62300000 14.011801732736334
62400000 12.036253911963433
62500000 11.289046711636475
6260

88700000 11.846074956770968
88800000 11.14893655761553
88900000 10.052972750908388
89000000 10.190269028824856
89100000 11.059774048402707
89200000 10.530265216177435
89300000 8.842402905103615
89400000 7.004352598040879
89500000 3.963923592989829
89600000 8.908981098583483
89700000 7.251164045395415
89800000 11.1115671339776
89900000 10.274918967488146
90000000 10.8161172275124
90100000 10.058041998582194
90200000 10.755822186293129
90300000 11.420402963635414
90400000 11.607839013108501
90500000 11.02703902792729
90600000 10.917157570635359
90700000 10.072399366170618
90800000 10.03869966297645
90900000 10.053955507659138
91000000 10.46268697221152
91100000 7.117064785917693
91200000 7.918874645459532
91300000 4.509315265063598
91400000 5.370702172326728
91500000 9.693555574292203
91600000 10.33412257360594
91700000 10.163408296542963
91800000 10.494863598758778
91900000 10.012946687320785
92000000 10.518479499402414
92100000 11.21498687421754
92200000 11.861630384814404
92300000 11.

118000000 6.358830782945183
118100000 8.008051331375983
118200000 7.93213653906256
118300000 7.635445368409855
118400000 7.832291422527892
118500000 7.956156112799943
118600000 8.272729703404076
118700000 9.12710461171269
118800000 8.804830803187292
118900000 8.153451510509365
119000000 7.942942625404625
119100000 7.992449667455166
119200000 8.048939362654513
119300000 8.276699140421663
119400000 7.6810107270832955
119500000 4.646813593075089
119600000 3.69603572481966
119700000 4.069616376604199
119800000 6.454473895303421
119900000 7.9223021026497324
120000000 8.052445296386749
120100000 7.330259594351362
120200000 7.969674328350044
120300000 7.825746448125452
120400000 8.18904081702421
120500000 8.951655014447132
120600000 8.805863393832286
120700000 8.231966627883779
120800000 7.670933908518289
120900000 7.846275519416868
121000000 7.971726723287871
121100000 7.774869327107039
121200000 7.444657759488711
121300000 4.691815635512959
121400000 3.433590113610814
121500000 4.7478311899

147300000 6.709246760212402
147400000 6.786154213169919
147500000 6.332158662338647
147600000 4.143179900481095
147700000 5.467841339309461
147800000 6.216335039897643
147900000 5.871784406900853
148000000 3.2385328997138507
148100000 6.215840136213676
148200000 5.9891734483291605
148300000 3.0301963406270116
148400000 6.214898468800313
148500000 6.207855875461349
148600000 6.687207238997936
148700000 7.130129807582732
148800000 6.7596423687257765
148900000 6.663404927761176
149000000 6.686824015287208
149100000 7.124510517489811
149200000 6.564877052547497
149300000 6.1723255997567446
149400000 3.380016637623638
149500000 6.169135515318049
149600000 6.139890746934785
149700000 4.0863567270223795
149800000 4.638857074546761
149900000 6.450701134463837
150000000 5.990757532683363
150100000 3.134452631524404
150200000 6.1093580119651465
150300000 6.276860786693007
150400000 6.595517970146623
150500000 6.667333003667785
150600000 6.596394113205434
150700000 7.084433000408915
150800000 6.4

176500000 4.310013153572397
176600000 4.334178854963992
176700000 4.473824873730034
176800000 4.711239702357411
176900000 2.59826187663577
177000000 4.680524803395241
177100000 4.821686164432026
177200000 4.766389605532411
177300000 4.538524168446292
177400000 4.843022939082411
177500000 2.5380227810726104
177600000 4.818994332669735
177700000 4.510807173338472
177800000 4.308294125205862
177900000 4.061594921993861
178000000 4.250490568468491
178100000 1.8568940201492474
178200000 3.935776631977866
178300000 4.208681591342091
178400000 4.755675541339626
178500000 4.1541331524024905
178600000 4.204629704032136
178700000 2.5039614344091414
178800000 4.407898320597976
178900000 4.567985125186288
179000000 4.855415038649679
179100000 4.5161858443794545
179200000 3.530858362547073
179300000 3.0073521376670205
179400000 4.252529222725898
179500000 3.9040146451481648
179600000 4.2419873084181114
179700000 3.8062951222870547
179800000 3.071461302410712
179900000 2.760123328939898
180000000 3.

205300000 2.602485446177294
205400000 2.2486156734229326
205500000 1.5095956134443878
205600000 1.4272649500668613
205700000 2.4864671535070504
205800000 2.542158096970151
205900000 2.4641490036159555
206000000 2.51865945615324
206100000 2.424640292918652
206200000 2.3378245632290775
206300000 2.43459999714909
206400000 1.3198215264941455
206500000 1.9644973545587494
206600000 2.412023387995806
206700000 2.2970487939672073
206800000 2.397034498076757
206900000 2.4219076614881687
207000000 2.3261554254926144
207100000 2.402611405876143
207200000 2.049768281990813
207300000 1.4998956487025321
207400000 1.3680345999275279
207500000 2.2087383893666566
207600000 2.2866979997779926
207700000 2.3336149930396908
207800000 2.1892753944623173
207900000 2.2228175059269866
208000000 2.1706416101463306
208100000 2.170285023012701
208200000 0.9536887890480459
208300000 2.239319662657473
208400000 2.118444547236896
208500000 2.1776176754977605
208600000 2.1172486517696645
208700000 2.309435262518512


233300000 0.09171651696020366
233400000 0.11644506611728668
233500000 0.0642032469909158
233600000 0.13168446524443891
233700000 0.08993556705334782
233800000 0.0802633408141401
best so far: 0
type: [1, 1, 1, 121, 1] 121
cases of this type: 1771561
233900000 0.05695340513751573
234000000 0.021089443134218455
234100000 0.02064280426306857
234200000 0.022433921627806296
234300000 0.021531542667359113
234400000 0.020769609263287653
234500000 0.01815655735078454
234600000 0.01690978317717711
234700000 0.019148850020670228
234800000 0.017979496050973733
234900000 0.012868012869000436
235000000 0.017647032572216458
235100000 0.016476516648100483
235200000 0.016257122910887
235300000 0.015708608495463926
235400000 0.01482222823855281
235500000 0.01440879645973444
235600000 0.00954326253391968
best so far: 0
type: [1, 1, 11, 1, 11] 121
cases of this type: 1771561
235700000 0.048491465933511654
235800000 0.04967080883614222
235900000 0.04619985771425566
236000000 0.03484902741554049
236100000 0